In [1]:

!pip install copulas
!pip install matplotlib

  Using cached numpy-1.22.4-cp39-cp39-win_amd64.whl (14.7 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.0
    Uninstalling numpy-1.23.0:

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
daal4py 2021.3.0 requires daal==2021.2.3, which is not installed.
numba 0.54.1 requires numpy<1.21,>=1.17, but you have numpy 1.22.4 which is incompatible.



      Successfully uninstalled numpy-1.23.0


In [2]:
!pip install jupyter_dash

In [3]:
pip install numpy --upgrade --ignore-installed


  Using cached numpy-1.23.0-cp39-cp39-win_amd64.whl (14.7 MB)
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
daal4py 2021.3.0 requires daal==2021.2.3, which is not installed.
scipy 1.7.1 requires numpy<1.23.0,>=1.16.5, but you have numpy 1.23.0 which is incompatible.
numba 0.54.1 requires numpy<1.21,>=1.17, but you have numpy 1.23.0 which is incompatible.


In [4]:
import numpy
print(numpy.__version__)

1.23.0


C:\Users\lpana\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\lpana\anaconda3\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
C:\Users\lpana\anaconda3\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [5]:
import matplotlib.pyplot as plt
import os
import pandas as pd
import numpy as np
from copulas.multivariate import GaussianMultivariate
from plotly.subplots import make_subplots
from copulas.univariate import Univariate

import plotly.graph_objects as go
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

C:\Users\lpana\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
C:\Users\lpana\AppData\Local\Temp/ipykernel_11040/427058258.py:12: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
C:\Users\lpana\AppData\Local\Temp/ipykernel_11040/427058258.py:13: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


In [6]:
def addPulp(name):
    data = pd.read_csv(name, header = None, index_col=False)
    data.dropna(axis = 0)
    data.drop(index=data.index[0], 
        axis=0, 
        inplace=True)
    
    data.columns = ('x', 'y')
    data['x']= pd.to_numeric(data['x'], errors = 'coerce')
    data['y'] = pd.to_numeric(data['y'], errors = 'coerce')
    newData = {}
    firstBin = data[data["x"] <= 6]
    firstBin = firstBin[firstBin["x"] >=0]
    firstBin = firstBin[firstBin["y"] <= 8]
    firstBin = firstBin[firstBin["y"] >= 0]
    
    secondBin = data[data["x"] <= 41]
    secondBin = secondBin[secondBin["x"] >=6]
    secondBin = secondBin[secondBin["y"] <= 6]
    secondBin = secondBin[secondBin["y"] >= 0]
    
    thirdBin = data[data["x"] <= 65]
    thirdBin = thirdBin[thirdBin["x"] >=40]
    thirdBin = thirdBin[thirdBin["y"] <= 12]
    thirdBin = thirdBin[thirdBin["y"] >= 5]
    
    bins = [firstBin, secondBin, thirdBin]
    dense = pd.DataFrame()
    for bin in bins:
        copula = GaussianMultivariate()
        copula.fit(bin)
        synthetic_data = copula.sample(int(len(bin)/len(data)*5000))
        synthetic_data = synthetic_data[synthetic_data["x"] < max(data['x'])+4]
        synthetic_data = synthetic_data[synthetic_data["x"] > min(data['x'])-1]
        synthetic_data = synthetic_data[synthetic_data["y"] < max(data['y'])+2]
        synthetic_data = synthetic_data[synthetic_data["y"] > min(data['y'])-1]
        dense = dense.append(synthetic_data, ignore_index=True)

    return {'Old': data, "Dense": dense}


In [7]:
def addCategorical(name):
    data = pd.read_csv(name, header = None, index_col=False)
    data.dropna(axis = 0)
    data.drop(index=data.index[0], 
        axis=0, 
        inplace=True)
    
    data.columns = ('x', 'y')
    data['x']= pd.to_numeric(data['x'], errors = 'coerce')
    data['y'] = pd.to_numeric(data['y'], errors = 'coerce')
    uniquex = data['x'].unique()
    DataFrameDict = {elem : pd.DataFrame() for elem in uniquex}

    for key in DataFrameDict.keys():
        DataFrameDict[key] = data[:][data['x'] == key]
    syntheticArray = []
    totalLength = (len(data.index))
    totalX = []
    totalY = []
    for key, value in DataFrameDict.items():
        univariate = Univariate()

        univariate.fit(value['y'])
        appendArray = []
        synthetic_data = univariate.sample(int(len(value.index)/totalLength*5000))
        dataX = [key]*len(synthetic_data)

        totalX = totalX + dataX
        totalY = totalY + synthetic_data.tolist()
    df = pd.DataFrame({'x': totalX,'y': totalY} ) 
    df = df[df["x"] <= max(data['x'])]
    df = df[df["x"] >= min(data['x'])]
    df = df[df["y"] <= max(data['y'])]
    df = df[df["y"] >= min(data['y'])]
    return {'Old': data, "Dense": df}


In [8]:
import random
def densify(df, x, y, exponent):
   
    xMore = []
    yMore = []
    
    x_norm = np.linalg.norm(df[x])
    y_norm = np.linalg.norm(df[y])

    df['x_norm']=[x_val/x_norm for x_val in df[x]]
    df['y_norm']=[y_val/y_norm for y_val in df[y]]
    
    for i in range(5000):
        
        j = random.randint(2, df.shape[0]-1)
        
        if j !=0 or j != 1:
          xMore.append(df['x_norm'][j] + np.random.laplace(0, (10**(-exponent))))
          yMore.append(df['y_norm'][j] + np.random.laplace(0, (10**(-exponent))))
    
    dense = pd.DataFrame()
    dense[x] = xMore
    dense[y] = yMore

    return dense
  
def jitterData(name, exponent):
  

  data = pd.read_csv(name, header = None, index_col=False)

  data.drop(index=data.index[0], 
        axis=0, 
        inplace=True)
  data.columns = ('x', 'y')
  data['x']= pd.to_numeric(data['x'], errors = 'coerce')
  data['y'] = pd.to_numeric(data['y'], errors = 'coerce')

  dense = densify(data, 'x', 'y', exponent)
  return {'Old': data, "Dense": dense}

In [9]:
def addMultipleCopulas2(name):
    data = pd.read_csv(name, header = None, index_col=False)
    data.dropna(axis = 0)
    data.drop(index=data.index[0], 
        axis=0, 
        inplace=True)
    
    data.columns = ('x', 'y')
    data['x']= pd.to_numeric(data['x'], errors = 'coerce')
    data['y'] = pd.to_numeric(data['y'], errors = 'coerce')
    newData = {}
    firstBin = data[data["x"] <= 18]
    firstBin = firstBin[firstBin["x"] >=0]
    
    secondBin = data[data["x"] <= 19]
    secondBin = secondBin[secondBin["x"] >=18]

    thirdBin =  data[data["x"] >=19]
    thirdBin = thirdBin.reset_index(drop=True)
    bins = [firstBin, secondBin, thirdBin]
    dense = pd.DataFrame()
    for bin in bins:
        
        if bin.equals(thirdBin):
            xMore = []
            yMore = []
            additionalPoints = int(len(bin)/len(data)*5000)
            for a in range(additionalPoints):
                j = random.randint(2, thirdBin.shape[0]-1)
                
                if j !=0 or j != 1:
                    xMore.append(thirdBin['x'][j] + np.random.laplace(0, (10**(-3))))
                    yMore.append(thirdBin['y'][j] + np.random.laplace(0, (10**(-3))))
            more = pd.DataFrame()
            more['x'] = xMore
            more['y'] = yMore

            dense = dense.append(more, ignore_index=True)
            
        else:
            copula = GaussianMultivariate()
            copula.fit(bin)
            synthetic_data = copula.sample(int(len(bin)/len(data)*5000))
       
        dense = dense.append(synthetic_data, ignore_index=True)

    return {'Old': data, "Dense": dense}

   
    

In [10]:
def sparsify(name):
    data = pd.read_csv(name, header = None, index_col=False)
    data.dropna(axis = 0)
    data.drop(index=data.index[0], 
        axis=0, 
        inplace=True)
    
    data.columns = ('x', 'y')
    data['x']= pd.to_numeric(data['x'], errors = 'coerce')
    data['y'] = pd.to_numeric(data['y'], errors = 'coerce')
    xMore = []
    yMore = []
    for a in range(5000):
        j = random.randint(2, data.shape[0]-1)

        if j !=0 or j != 1:
            xMore.append(data['x'][j])
            yMore.append(data['y'][j])
    dense = pd.DataFrame()
    dense['x'] = xMore
    dense['y'] = yMore
    return {'Old': data, "Dense": dense}


In [11]:
def addExact(name):
    data = pd.read_csv(name, header = None, index_col=False)
    data.dropna(axis = 0)
    
    data.drop(index=data.index[0], 
        axis=0, 
        inplace=True)
    
    data.columns = ('x', 'y')
    data['x']= pd.to_numeric(data['x'], errors = 'coerce')
    data['y'] = pd.to_numeric(data['y'], errors = 'coerce')
    xMore = []
    yMore = []
    for a in range(5000):
        j = random.randint(2, data.shape[0]-1)

        if j !=0 or j != 1:
            xMore.append(data['x'][j])
            yMore.append(data['y'][j])
    dense = pd.DataFrame()
    dense['x'] = xMore
    dense['y'] = yMore
    return {'Old': data, "Dense": dense}

In [12]:
def addCategoricalY(name):
    data = pd.read_csv(name, header = None, index_col=False)
    data.dropna(axis = 0)
    data.drop(index=data.index[0], 
        axis=0, 
        inplace=True)
    
    data.columns = ('x', 'y')
    data['x']= pd.to_numeric(data['x'], errors = 'coerce')
    data['y'] = pd.to_numeric(data['y'], errors = 'coerce')
    uniquey = data['y'].unique()
    DataFrameDict = {elem : pd.DataFrame() for elem in uniquey}

    for key in DataFrameDict.keys():
        DataFrameDict[key] = data[:][data['y'] == key]

    syntheticArray = []
    totalLength = (len(data.index))
    totalX = []
    totalY = []
    for key, value in DataFrameDict.items():
        univariate = Univariate()

        univariate.fit(value['x'])
        appendArray = []
        synthetic_data = univariate.sample(int(len(value.index)/totalLength*5000))

        dataY = [key]*len(synthetic_data)

        totalX = totalX + synthetic_data.tolist()
        totalY = totalY + dataY
    df = pd.DataFrame({'x': totalX,'y': totalY} ) 
    df = df[df["x"] <= max(data['x'])]
    df = df[df["x"] >= min(data['x'])]
    df = df[df["y"] <= max(data['y'])]
    df = df[df["y"] >= min(data['y'])]
    return {'Old': data, "Dense": df}

In [13]:
def useCopula(name):
    
    data = pd.read_csv(name, header = None, index_col=False)
    data = data.dropna(axis = 0)
    data = data.reset_index(drop=True)
    data.drop(index=data.index[0], 
        axis=0, 
        inplace=True)
    
    data.columns = ('x', 'y')
    data['x']= pd.to_numeric(data['x'], errors = 'coerce')
    data['y'] = pd.to_numeric(data['y'], errors = 'coerce')
    copula = GaussianMultivariate()
    copula.fit(data)
    synthetic_data = copula.sample(5000)
    synthetic_data = synthetic_data[synthetic_data["x"] < max(data['x'])]
    synthetic_data = synthetic_data[synthetic_data["x"] > min(data['x'])]
    synthetic_data = synthetic_data[synthetic_data["y"] < max(data['y'])]
    synthetic_data = synthetic_data[synthetic_data["y"] > min(data['y'])]
    return {'Old': data, "Dense": synthetic_data}
    

In [14]:
def addMultipleCopulas18(name):
    data = pd.read_csv(name, header = None, index_col=False)
    data.dropna(axis = 0)
    data.drop(index=data.index[0], 
        axis=0, 
        inplace=True)
    
    data.columns = ('x', 'y')
    data['x']= pd.to_numeric(data['x'], errors = 'coerce')
    data['y'] = pd.to_numeric(data['y'], errors = 'coerce')
    newData = {}
    firstBin = data[data["y"] <= 2]

    
    secondBin = data[data["y"] >= 2]


    bins = [firstBin, secondBin]
    dense = pd.DataFrame()
    for bin in bins:
        xMore = []
        yMore = []
        additionalPoints = int(len(bin)/len(data)*5000)
        copula = GaussianMultivariate()
        copula.fit(bin)
        synthetic_data = copula.sample(int(len(bin)/len(data)*5000))
       
        dense = dense.append(synthetic_data, ignore_index=True)

    return {'Old': data, "Dense": dense}

   

In [15]:
chartDictionary = {'0':['./finalData/0/gilgais.csv', 2.5], '1':['./finalData/1/MCAS.csv', 3], 
                   '2':['./finalData/2/Hedonic.csv'], '3': ['./finalData/3/HC.csv'],
                  '4':['./finalData/4/MedExp.csv'], '5':['./finalData/5/Cars93.csv'],
                  '6':['./finalData/6/Cars93.csv', 3.5], '7':['./finalData/7/toxicity.csv'],
                  '8':['./finalData/8/VietNamH.csv'], '9':['./finalData/9/Garch.csv', 4],
                  '10':['./finalData/10/Fishing.csv'], '11':['./finalData/11/Forbes2000.csv'],
                  '12':['./finalData/12/incomeinequality.csv'],'13':['./finalData/13/volcano.csv',3],
                  '14':['./finalData/14/cache.csv'], '15':['./finalData/15/Fatality.csv'],
                  '16':['./finalData/16/DoctorContacts.csv'],  '17':['./finalData/17/Sitka.csv'],
                  '18':['./finalData/18/iris.csv'], '19':['./finalData/19/radarImage.csv']}

In [16]:
denseCharts = {}
for key,value in chartDictionary.items():
    if key == '0':
        dictionary = jitterData(value[0], value[1])
    if key == '1':
        dictionary = jitterData(value[0], value[1])
    if key == '2':
        dictionary = addMultipleCopulas2(value[0])
    if key == '3':
        dictionary = useCopula(value[0])
    if key == '4':
        dictionary = sparsify(value[0])
    if key == '5':
        dictionary = useCopula(value[0])
    if key == '6':
        dictionary = jitterData(value[0], value[1])
    if key == '7':
        dictionary = useCopula(value[0])
    if key == '8':
        dictionary = sparsify(value[0])
    if key == '9':
        dictionary = jitterData(value[0], value[1])
    if key == '10':
        dictionary = addExact(value[0])
    if key == '11':
        dictionary = useCopula(value[0])
    if key == '12':
        dictionary = useCopula(value[0])
    if key == '13':
        dictionary = jitterData(value[0], value[1])
    if key == '14':
        dictionary = addCategorical(value[0])
    if key == '15':
        dictionary = addExact(value[0])
    if key == '16':
        dictionary = sparsify(value[0])
    if key == '17':
        dictionary = addCategoricalY(value[0])
    if key == '18':
        dictionary = addMultipleCopulas18(value[0])
    if key == '19':
        dictionary = addExact(value[0])

  
    denseCharts[key] = dictionary

C:\Users\lpana\anaconda3\lib\site-packages\scipy\stats\_continuous_distns.py:5320: RuntimeWarning: divide by zero encountered in divide
  return c**2 / (c**2 - n**2)
C:\Users\lpana\anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:2606: RuntimeWarning: invalid value encountered in multiply
  Lhat = muhat - Shat*mu
C:\Users\lpana\anaconda3\lib\site-packages\scipy\stats\_continuous_distns.py:639: RuntimeWarning: invalid value encountered in sqrt
  sk = 2*(b-a)*np.sqrt(a + b + 1) / (a + b + 2) / np.sqrt(a*b)
C:\Users\lpana\anaconda3\lib\site-packages\scipy\optimize\minpack.py:175: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)
C:\Users\lpana\anaconda3\lib\site-packages\scipy\stats\_continuous_distns.py:5311: RuntimeWarning: divide by zero encountered in power
  return cd2*x**(c-1)
C:\Users\lpana\anaconda3\lib\site-packages\scipy\optimize\optimize.py:282: RuntimeW

In [17]:
def getDataFrame(name):
  return denseCharts[name]

In [18]:
def makeFig(name):
    dictionary = getDataFrame(name)
    fig = make_subplots(rows=2, cols=2)
    oldScatter = go.Scatter(
        x = dictionary['Old']['x'], y=  dictionary['Old']['y'], mode="markers", marker_size = 2, name="Original"
    )
    denseScatter = go.Scatter(
        x = dictionary['Dense']['x'], y=  dictionary['Dense']['y'], mode="markers", marker_size = 2, name = "Points Added"
    )
    fig.update_traces(marker_size=.1)
    HOld, xedges, yedges = np.histogram2d(dictionary['Old']['x'], dictionary['Old']['y'], bins = 64)
    HOld = HOld.T
    
    HDense, xedges, yedges = np.histogram2d(dictionary['Dense']['x'], dictionary['Dense']['y'], bins = 64)
    HDense = HDense.T
    oldHeat = go.Heatmap(x = xedges, y = yedges, z=HOld, colorscale='gray', showscale=False, reversescale= True)
    denseHeat = go.Heatmap(x = xedges, y = yedges, z = HDense, colorscale='gray', showscale=False, reversescale= True)
    
    fig.add_trace(oldScatter, row=1, col=1)
    fig.add_trace(denseScatter, row=1, col=2)
    fig.add_trace(oldHeat, row =2, col =1)
    fig.add_trace(denseHeat, row =2, col =2)
    fig.update_xaxes(visible=False)
    fig.update_yaxes(visible=False)
    fig.update_layout(height=800, width=800, title_text=name)
    return fig

In [19]:
dataFrameDictionary = getDataFrame('0')

app = JupyterDash(__name__)
app.layout = html.Div([
#     html.H1("Environmental Data"),
    dcc.Graph(id='graph'),
    html.Div([
        "charts",
        dcc.Dropdown(
            id='chart-dropdown', clearable=False,
            value='0', options=[
                {'label': c, 'value': c}
                for c in chartDictionary.keys()
            ]),
    ]),
])



@app.callback(
    Output('graph', 'figure'),
    Input('chart-dropdown', 'value'))
def update_figure_old(selected_chart):
    return makeFig(selected_chart)

# if __name__ == '__main__':
#     app.run_server(use_reloader=False, debug=False, host='localhost', port=1050)
if __name__ == '__main__':
    app.run_server(debug=True)

Dash app running on http://127.0.0.1:8050/


In [20]:
!pip install fpdf

In [21]:
!pip install -U kaleido

In [27]:
from fpdf import FPDF
pdf = FPDF()

for key,value in denseCharts.items():
    pdf.add_page()
    fig = makeFig(key)
    fig.write_image("./images/"+key+".png")
    pdf.image("./images/"+key+".png",10,10,175,175)
pdf.output("./yourfile.pdf", "F")

''